**Classification tests**

In [30]:
from imports import *
from sklearn import datasets
from classes import NeuralNetwork, GradientDescent
#from functions import mse, cross_entropy, mse_der, sigmoid, sigmoid_der, reLU, ReLU_der, identity, identity_der, accuracy, cross_entropy, polynomial_features
from functions import *
from autograd import grad
import classes
import functions
import importlib
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

importlib.reload(classes) 
importlib.reload(functions)


<module 'functions' from 'c:\\Users\\Jogga\\Documents\\H25\\FYS-STK4155\\FYS-STK4155_coursework\\Project-2\\Code\\functions.py'>

Testing the neural net for classification. Plotting accuracy as function of depth and number of nodes for multiclass. Need to tweak params to get the best accuracies. I believe the NN is very prone to changes in performance best on these...

In [31]:

import seaborn as sns


def onehot(y, n=None):
    y = np.asarray(y, dtype=int).ravel()
    n = np.max(y) + 1 if n is None else n
    m = np.zeros((y.size, n))
    m[np.arange(y.size), y] = 1
    return m


mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X = mnist.data.astype(np.float32) / 255.0
y = mnist.target.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)
Y_train, Y_test = onehot(y_train), onehot(y_test)

layers_list = [0, 1, 2, 3]
nodes_list = [5, 10, 25, 50]
acc = np.zeros((len(layers_list), len(nodes_list)))

for i, L in enumerate(layers_list):
    for j, N in enumerate(nodes_list):
        if L == 0:
            layers, acts, ders = [10], [softmax], [softmax_der]
        else:
            layers = [N]*L + [10]
            acts = [sigmoid]*L + [softmax]
            ders = [sigmoid_der]*L + [softmax_der]

        Classifier = NeuralNetwork(X_train.shape[1], layers, acts, ders, cross_entropy, cross_entropy_der)
        Classifier.train_SGD(X_train, Y_train, epochs=10, learning_rate=0.2, batch_size=100)
        predictions = Classifier.feed_forward(X_test)
        y_pred_label = np.argmax(predictions, axis=1)
        accuracy = np.mean(y_test == y_pred_label)
        acc[i,j] = accuracy
        print(f"Accuracy for {i} layers and {j} nodes: {accuracy}")

import os

save_dir = os.path.join(os.path.dirname(os.getcwd()), "Figures")

os.makedirs(save_dir, exist_ok=True)

fig, ax = plt.subplots(figsize=(8, 5))
sns.heatmap(acc, annot=True, fmt=".2f", cmap="viridis", ax=ax)
ax.set_xlabel("Nodes per Hidden Layer", fontsize=15)
ax.set_ylabel("Number of Hidden Layers", fontsize=15)
ax.set_title("Test Accuracy Heatmap", fontsize=17)
ax.legend()


save_path = os.path.join(save_dir, "HM_classification.png")
fig.savefig(save_path, dpi=300, bbox_inches='tight')


(1,)


AttributeError: 'NeuralNetwork' object has no attribute 'lam'